# Суммаризация:
1) mBART, RuT5
2) Делаем общую суммарзацию
3) делаем по странично

# Без препроуцессинга

In [54]:
import PyPDF2
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm  # Добавляем tqdm

In [55]:
# Функция для чтения страниц из PDF файла
def extract_pages_from_pdf(pdf_path):
    pages = []
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page_text = pdf_reader.pages[page_num].extract_text()
            pages.append(page_text)
    return pages

In [56]:
# Чтение PDF файла и выполнение суммаризации для каждой страницы
pdf_path = 'D:\\ProgPrj\\vsProjects\\echo_hack\\if_llm\\pipe_1\\datasets\\aprox\\росатом_маяк.pdf'  # Замените на путь к вашему PDF файлу
pages = extract_pages_from_pdf(pdf_path)

In [57]:
pages

['ОТЧЕТ\nпо экологической безопасности \nФГУП «ПО «Маяк»\nза 2022  год\nОзерск – 2023Федеральное государственное унитарное предприятие\n«Производственное объединение « Маяк»\n',
 'УДК 621.039(470.4/5):502.13\nББК 31.4(2Рос-4Че)-4О-88\nОтчет по экологической безопасности ФГУП «ПО «Маяк» за 2022 год / Госкорпорация \n«Росатом»; ФГУП «ПО «Маяк». – Озерск: РИЦ ВРБ; Типография ФГУП «ПО «Маяк», 2023. – 56 с.\nОтчет по экологической безопасности предприятия, входящего в состав Государственной \nкорпорации по атомной энергии «Росатом», характеризует важнейшие направления его приро-доохранной деятельности в 2022 году.\nОтчет представляет документально подтвержденные сведения о воздействии производ-\nственной деятельности предприятия на окружающую среду, а также об экологическом контроле и мероприятиях по сокращению воздействия производственных процессов на окружающую среду.\nЦель отчета – информировать население, экологические организации, научные и социаль-\nные институты, органы местного само

In [58]:
# Функция для суммаризации текста с использованием RuT5
def summarize_text(text, model, tokenizer):
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    
    summary_ids = model.generate(input_ids, 
                                 max_length=150, 
                                 min_length=40, 
                                 length_penalty=2.0, 
                                 num_beams=4, 
                                 early_stopping=True)
    
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [7]:
# Инициализация модели и токенизатора RuT5
model_name = "IlyaGusev/rut5_base_sum_gazeta"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

C:\Users\Geo\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
C:\Users\Geo\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This s

In [59]:
# Добавляем tqdm для отслеживания прогресса
summaries = []
for page in tqdm(pages, desc="Summarizing pages", unit="page"):
    summary = summarize_text(page, model, tokenizer)
    summaries.append(summary)

Summarizing pages: 100%|██████████| 56/56 [05:47<00:00,  6.21s/page]


In [9]:
# Вывод результатов
for i, summary in enumerate(summaries):
    print(f"Summary of page {i + 1}:")
    print(summary)
    print("-" * 50)

Summary of page 1:
ФГУП «ПО «Маяк» за 2022 год Озерск – 2023Федеральное государственное унитарное предприятие «Производственное объединение «Маяк» за 2022 год Озерск – 2023Федеральное государственное унитарное предприятие «Производственное объединение «Маяк» за 2022 год Озерск – 2023Федеральное государственное унитарное предприятие «Производственное объединение «Маяк» за 2022 год Озерск – 2023Федеральное государственное унитарное предприятие «Производственное объединение «Маяк» за 2022 год Озерск – 2023Федеральное государственное унитарное предприятие «
--------------------------------------------------
Summary of page 2:
Государственная корпорация по атомной энергии «Росатом» опубликовала отчет по экологической безопасности ФГУП «ПО «Маяк» за 2022 год. Отчет представляет документально подтвержденные сведения о воздействии производственных процессов на окружающую среду и мерах по сокращению воздействия производственных процессов на окружающую среду.
------------------------------------